In [4]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import pandas as pd



In [5]:
artisti = []
limit = 50
after = 0
results = sp.current_user_saved_tracks(limit=limit, offset=after)

while True:
    results = sp.current_user_saved_tracks(limit=limit, offset=after)
    print("Limite: ", limit, ", Offset: ", after, ", Numero di risultati: ", len(results['items']))

    for item in results['items']:
        track = item["track"]
        if track["type"] == "track":
            for artist in track["artists"]:
                if not list(filter(lambda x: x["uri"] == artist["uri"], artisti)):
                    artisti.append({"uri":  artist["uri"], "name": artist["name"]})
    
    after += limit
    if len(results['items']) < limit:   
        break

In [2]:
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]


def artist_filter(genre, artists):
    results = []
    artists_id = [artist["uri"] for artist in artists]


    for artists_id_chunk in divide_chunks(artists_id, 50):
        artists_more_info = sp.artists(artists_id_chunk)

        for artist in artists_more_info["artists"]:
            for music_genre in artist["genres"]:
                if genre in music_genre:
                    result = {}
                    result["uri"], result["name"], result["followers"] = artist["uri"], artist["name"], artist["followers"]["total"]
                    results.append(result)
                    break

    return results

In [ ]:
# Filtro per rapper italiani
artisti = artist_filter("italian", artisti)


with open('artisti_liberia.json', 'w') as outfile:
    json.dump(artisti, outfile, indent=4)

In [ ]:
import json

with open('artisti_liberia.json') as json_file:
    artisti = json.load(json_file)

In [ ]:
import time

def flatten(t):
    return [item for sublist in t for item in sublist]

def get_artists_from_artists(artist_uri, missing_steps):
    print("Artista corrente:", artist_uri)
    if missing_steps > 0:
        try:
            artisti = artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])
        except Exception as e:
            print(e)
            time.sleep(5)
            artisti = artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])

        partial_result = flatten([get_artists_from_artists(artista["uri"], missing_steps - 1) for artista in artisti])
    
        return artisti + partial_result
    else:
        try:
            return artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])
        except Exception as e:
            print(e)
            time.sleep(5)
            return artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])

recursive_artist = get_artists_from_artists(artisti[0]["uri"], 3)
recursive_artist_no_dupl = []

for x in recursive_artist:
    if x not in recursive_artist_no_dupl:
        recursive_artist_no_dupl.append(x)

with open('artisti_ricorsivi.json', 'w') as json_file:
    json.dump(recursive_artist_no_dupl, json_file, indent=4)

In [69]:
len(recursive_artist_no_dupl)

183